LIVROS - CSV

In [38]:
!pip install -q pandas

In [39]:
import pandas as pd
import os

In [40]:
df = pd.read_csv('../data/livros.csv')
df.head()

,Book,Author(s),Original language,First published,Approximate sales in millions,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga


In [41]:
df.shape

(174, 6)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Book                           174 non-null    object 
 1   Author(s)                      174 non-null    object 
 2   Original language              174 non-null    object 
 3   First published                174 non-null    int64  
 4   Approximate sales in millions  174 non-null    float64
 5   Genre                          118 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 8.3+ KB


In [43]:
df.columns

Index(['Book', 'Author(s)', 'Original language', 'First published',
       'Approximate sales in millions', 'Genre'],
      dtype='object')

In [44]:
renamed_columns = {
    'Book': 'book',
    'Author(s)': 'author',
    'Original language': 'org_lang',
    'First published': 'year_published',
    'Approximate sales in millions': 'sales',
    'Genre': 'genre'
}

In [45]:
df.rename(columns=renamed_columns, inplace=True)

In [46]:
df.head()

,book,author,org_lang,year_published,sales,genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga


In [47]:
df.isnull().sum()

book               0
author             0
org_lang           0
year_published     0
sales              0
genre             56
dtype: int64

In [48]:
df["genre"] = df["genre"].fillna("Unknown")

In [49]:
df["genre"].isnull().sum()

np.int64(0)

In [50]:
df["author"].value_counts()

author
J. K. Rowling                 7
Dan Brown                     3
Suzanne Collins               3
Roald Dahl                    3
George Orwell                 2
                             ..
Eliyahu M. Goldratt           1
Ray Bradbury                  1
Frank McCourt                 1
Mohandas Karamchand Gandhi    1
Helen Fielding                1
Name: count, Length: 157, dtype: int64

In [51]:
df_authors = pd.DataFrame(df["author"].unique(), columns=["author"])
df_authors["author_id"] = df_authors.index + 1

In [52]:
df_authors.head()

,author,author_id
0,Charles Dickens,1
1,Antoine de Saint-Exupéry,2
2,J. K. Rowling,3
3,Agatha Christie,4
4,Cao Xueqin,5


In [53]:
df = df.merge(df_authors, on="author", how="left")
df.head()

,book,author,org_lang,year_published,sales,genre,author_id
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction,1
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella,2
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery,4
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga,5


In [54]:
authors = pd.DataFrame(df['author'].unique(), columns=['name'])

In [55]:
with open("../sql/inserts/authors.sql", "w", encoding="utf-8") as f:
  for _, row in authors.iterrows():
    name = row['name'].replace("'", "''")
    f.write(f"INSERT INTO authors (name) VALUES ('{name}');\n")

In [56]:
df_genre = pd.DataFrame(df["genre"].unique(), columns=["genre"])
df_genre["genre_id"] = df_genre.index + 1

In [57]:
df = df.merge(df_genre, on="genre", how="left")
df.head()

,book,author,org_lang,year_published,sales,genre,author_id,genre_id
0,A Tale of Two Cities,Charles Dickens,English,1859,200.0,Historical fiction,1,1
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,200.0,Novella,2,2
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120.0,Fantasy,3,3
3,And Then There Were None,Agatha Christie,English,1939,100.0,Mystery,4,4
4,Dream of the Red Chamber (紅樓夢),Cao Xueqin,Chinese,1791,100.0,Family saga,5,5


In [58]:
genres = pd.DataFrame(df["genre"].unique(), columns=["genre"])

In [59]:
!pip install -q deep-translator

In [60]:
from deep_translator import GoogleTranslator

genres["genre_pt"] = genres["genre"].apply(
    lambda x: GoogleTranslator(source="auto", target="pt").translate(x)
)
genres.head()

,genre,genre_pt
0,Historical fiction,Ficção histórica
1,Novella,Novella
2,Fantasy,Fantasia
3,Mystery,Mistério
4,Family saga,Saga da família


In [61]:
genres["genre_pt"] = genres["genre_pt"].replace("Novella", "Novela")
genres["genre_pt"] = genres["genre_pt"].str.lower()
genres["genre"] = genres["genre"].str.lower()

In [62]:
with open("../sql/inserts/genres.sql", "w", encoding="utf-8") as f:
  for index, row in genres.iterrows():
    genre = row["genre"].replace("'", "''")
    genre_pt = row["genre_pt"].replace("'", "''")
    f.write(f"INSERT INTO genres (genre, genre_pt) VALUES ('{genre}', '{genre_pt}');\n")

In [63]:
df["book_id"] = df.index + 1

In [64]:
with open("../sql/inserts/books.sql", "w", encoding="utf-8") as f:
  for _, row in df.iterrows():
    book_name = row["book"].replace("'", "''")
    org_lang = row["org_lang"].replace("'", "''")
    year_published = int(row["year_published"])
    sales = float(row["sales"])
    author_id = int(row["author_id"])
    genre_id = int(row["genre_id"])
    sql = (f"INSERT INTO books (book_name, org_lang, year_published, sales, author_id, genre_id) "
            f"VALUES ('{book_name}', '{org_lang}', '{year_published}', '{sales:.2f}', '{author_id}', '{genre_id}');\n"
    )
    f.write(sql)

LIVROS - API

In [65]:
api = "https://raw.githubusercontent.com/guilhermeonrails/datas-csv/refs/heads/main/comentarios.json"
df_coments = pd.read_json(api)
df_coments.head()

,id,livro,nome,sobrenome,comentario
0,1,The Dukan Diet,Danielle,Johnson,Achei a história envolvente e bem escrita.
1,2,"The Lion, the Witch and the Wardrobe",Joshua,Walker,Esperava mais do enredo.
2,3,James and the Giant Peach,Jill,Rhodes,Personagens cativantes e trama surpreendente.
3,4,The Very Hungry Caterpillar,Patricia,Miller,Achei a história envolvente e bem escrita.
4,5,Bridget Jones's Diary,Robert,Johnson,Um dos melhores que já li.


In [66]:
df_coments.shape

(10000, 5)

In [67]:
df_coments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          10000 non-null  int64 
 1   livro       10000 non-null  object
 2   nome        10000 non-null  object
 3   sobrenome   10000 non-null  object
 4   comentario  10000 non-null  object
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [68]:
renamed_columns = {
    'livro': 'book',
    'nome': 'name',
    'sobrenome': 'last_name',
    'comentario': 'coment'
}
df_coments.rename(columns=renamed_columns, inplace=True)

In [69]:
df_coments.isnull().sum()

id           0
book         0
name         0
last_name    0
coment       0
dtype: int64

In [70]:
df_coments = df_coments.merge(df[["book"]].reset_index().rename(columns={"index": "book_id"}), on="book", how="left")
df_coments.head()

,id,book,name,last_name,coment,book_id
0,1,The Dukan Diet,Danielle,Johnson,Achei a história envolvente e bem escrita.,163
1,2,"The Lion, the Witch and the Wardrobe",Joshua,Walker,Esperava mais do enredo.,6
2,3,James and the Giant Peach,Jill,Rhodes,Personagens cativantes e trama surpreendente.,62
3,4,The Very Hungry Caterpillar,Patricia,Miller,Achei a história envolvente e bem escrita.,35
4,5,Bridget Jones's Diary,Robert,Johnson,Um dos melhores que já li.,173


In [71]:
print(f"{df_coments['book_id'].max()} {df_coments['book_id'].min()}")

173 0


In [72]:
df_coments["book_id"] += 1

In [73]:
def format_value(value):
  if pd.isna(value):
    return "NULL"
  elif isinstance(value, str):
    value = value.replace("'", "''")
    return f"'{value}'"
  else:
    return str(value)

In [74]:
with open("../sql/inserts/coments.sql", "w", encoding="utf-8") as f:
  for _, row in df_coments.iterrows():
    values = (
        format_value(row["book_id"]),
        format_value(row["name"]),
        format_value(row["last_name"]),
        format_value(row["coment"])
    )
    sql = f"INSERT INTO coments (book_id, name, last_name, coment) VALUES ({', '.join(values)});\n"
    f.write(sql)